# Catchment delineation
## Juncal Norte - august, 2022
#### Paul Sandoval Quilodrán - https://github.com/SQPaul

In [1]:
#Import packages
from osgeo import gdal, gdalconst
from pcraster import * 
from os import chdir, getcwd

### Change directory and define function

In [2]:
#Change directory
f_dir = "/home/phi/Desktop/Projects/Juncal_norte/GIS"
chdir(f_dir)

#Create function to convert from tif to map
def ConvertToPCRaster (src_fname,dst_fname,ot,VS):
    src_ds = gdal.Open(src_fname)
    dst_ds = gdal.Translate(dst_fname, src_ds, format="PCRaster", outputType=ot, metadataOptions=VS)
    src_ds = None
    dst_ds = None

### Fill dem

In [3]:
#1 Convert dem and fill it
ConvertToPCRaster("dem28.tif","dem.map",gdalconst.GDT_Float32,"VS_SCALAR")
dem = readmap("dem.map")
demf = lddcreatedem(dem,1e31,1e31,1e31,1e31)
report(demf,"demf.map")
aguila(demf)

### Local Drain Direction (ldd) & Accuflux

In [4]:
#2 Create local drain direction and river network
ldd = lddcreate(demf,1e31,1e31,1e31,1e31)
report(ldd,"ldd.map")
accuflux = accuflux(ldd,1)
report(accuflux,"accuflux.map")
aguila(accuflux)

### Strahler & reparir ldd

In [3]:
#3 Analyze strahler order NO LO HICE!!
strahler = streamorder("ldd.map")
aguila(strahler)
report(strahler,"strahler.map")

##Repair ldd
#lddrep = lddrepair(ldd)
#report(lddrep,"lddrep.map")

### Outflows

In [3]:
#4 Define outputs
#4.1 In this step check if the outputs basins are over the accuflux (river network)
#4.2 Convert outputs to raster (points with attr=ID), (ID natural numbers)
#4.3 Convert to pcraster and nominal.map
ConvertToPCRaster("stations.tif","stations.map",gdalconst.GDT_Float32,"VS_SCALAR")
outflow = nominal("stations.map")
report(outflow,"outflow.map")

### Catchment & subcatchments

In [ ]:
#5 Delineate basin and subbasins
catchment = catchment("ldd.map","outflow.map")
report(catchment,"catchment.map")
subcatchment = subcatchment("ldd.map","outflow.map")
report(subcatchment,"subcatchment.map")
aguila(catchment)